**Step 1: Import Necessary Libraries**

In [8]:
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import plt
import yfinance as yf
import scipy.stats as stats
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.tsa.stattools import adfuller
from statsmodels.distributions.empirical_distribution import ECDF
from scipy.stats import kendalltau, pearsonr, spearmanr
from scipy.optimize import minimize
from scipy.integrate import quad
import sys
from collections import deque


plt.style.use('seaborn')
%matplotlib inline

<ipython-input-8-ab1be74e3760>:19: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


**Step 2: Define 'PairSelection' class**

In [16]:
class PairSelection():
    
    def __init__(self):
        self.lst_ticket1 = []
        self.lst_ticket2 = []
        self.final_lst = []
    
    def testStationary(self, data):
        adftest = adfuller(data)
        result = pd.Series(adftest[0:4], index = ['Test Statistic', 'p-value', 'Lags Used', 'Number of Observations Used'])
        for key,value in adftest[4].items():
            result['Critical Value(%s)' % key] = value
        result = pd.DataFrame(result)
        
        return result
    
    def pair_selection_helper_1(self, symbol_list):
        # select the designated pairs
        tau_ = 0
        
        for i in range(len(symbol_list) - 1):
            ticket_1 = yf.download(symbol_list[i], start = '1900-01-01', end = '2023-01-18', progress = False)
            for j in range(i + 1, len(symbol_list)):
                # Get the Actual Data in "symbol_list" (from here we all set uniform dates for simplicity)
                ticket_2 = yf.download(symbol_list[j], start = '1900-01-01', end = '2023-01-18', progress = False)
                
                # Process the raw data
                ticket1 = ticket_1[['Close']]
                ticket2 = ticket_2[['Close']]
                Ticket1 = ticket1.rename(columns = {'Close': 'ticket1'})
                Ticket2 = ticket2.rename(columns = {'Close': 'ticket2'})
                prices = pd.concat([Ticket1, Ticket2], axis = 1)
                
                price = prices.dropna()
                
                # Conduct the Cointegration Test
                baseline_regression_model = ols('ticket2 ~ ticket1', data = price).fit()
                beta_0 = baseline_regression_model.params[0]
                beta_1 = baseline_regression_model.params[1]
                
                coint_residual = price['ticket2'] - beta_0 - beta_1 * price['ticket1']
                adf_test_result = testStationary(coint_residual)
                new_col = [symbol_list[j] + '&' + symbol_list[i]]
                adf_test_result.columns = new_col
                adf_test_result = adf_test_result.T
                p_value = float(adf_test_result['p-value'])
                
                if p_value < 0.05:
                    self.lst_ticket1.append((symbol_list[i], symbol_list[j]))
                    
        return self.lst_ticket1
        
    def pair_selection_helper_2(self, symbol_list):
        # select the designated pairs
        tau_ = 0
        
        for i in range(len(symbol_list) - 1):
            for j in range(i + 1, len(symbol_list)):
                # Get the Actual Data in "symbol_list" (from here we all set uniform dates for simplicity)
                ticket_1 = yf.download(symbol_list[i], start = '1900-01-01', end = '2023-01-18', progress = False)
                ticket_2 = yf.download(symbol_list[j], start = '1900-01-01', end = '2023-01-18', progress = False)
                
                # Process the raw data
                ticket1 = ticket_1[['Close']]
                ticket2 = ticket_2[['Close']]
                Ticket1 = ticket1.rename(columns = {'Close': 'ticket1'})
                Ticket2 = ticket2.rename(columns = {'Close': 'ticket2'})
                prices = pd.concat([Ticket1, Ticket2], axis = 1)
                
                price = prices.dropna()
                
                # Conduct the Cointegration Test
                baseline_regression_model = ols('ticket1 ~ ticket2', data = price).fit()
                beta_0 = baseline_regression_model.params[0]
                beta_1 = baseline_regression_model.params[1]
                
                coint_residual = price['ticket1'] - beta_0 - beta_1 * price['ticket2']
                adf_test_result = testStationary(coint_residual)
                new_col = [symbol_list[j] + '&' + symbol_list[i]]
                adf_test_result.columns = new_col
                adf_test_result = adf_test_result.T
                p_value = float(adf_test_result['p-value'])
                
                if p_value < 0.05:
                    self.lst_ticket2.append((symbol_list[i], symbol_list[j]))
                    
        return self.lst_ticket2
    
    def lst_process(self):
        for symbol in self.lst_ticket1:
            if symbol in self.lst_ticket2:
                self.final_lst.append(symbol)
                
        return self.final_lst
    
    def pair_selection(self):
        # select the designated pairs
        tau_ = 0
        
        for i in range(len(self.final_lst)):
            # Get the Actual Data in "symbol_list" (from here we all set uniform dates for simplicity)
            ticket_1 = yf.download(self.final_lst[i][0], start = '1900-01-01', end = '2023-01-18', progress = False)
            ticket_2 = yf.download(self.final_lst[i][1], start = '1900-01-01', end = '2023-01-18', progress = False)
            
            # Process the raw data
            ticket1 = ticket_1[['Close']]
            ticket2 = ticket_2[['Close']]
            Ticket1 = ticket1.rename(columns = {'Close': symbol_list[i][0]})
            Ticket2 = ticket2.rename(columns = {'Close': symbol_list[i][1]})
            prices = pd.concat([Ticket1, Ticket2], axis = 1)
            
            # Calculate the returns
            returns = np.log(prices).diff().dropna()
            
            # Calculate the kendall rank correlation for this particular pair
            tau = stats.kendalltau(returns[symbol_list[i][0]], returns[symbol_list[i][1]])[0]
            
            if tau > tau_:
                tau_ = tau
                lst_returns = symbol_list[i]
                
        return [lst_returns, tau_] 

In [18]:
DWCF_df = yf.download('^DWCF', start = '1900-01-01', end = '2023-01-19', progress = False)

In [19]:
DWCF_df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1970-12-31,830.270020,830.270020,830.270020,830.270020,830.270020,0
1971-01-29,873.309998,873.309998,873.309998,873.309998,873.309998,0
1971-02-26,885.369995,885.369995,885.369995,885.369995,885.369995,0
1971-03-31,923.130005,923.130005,923.130005,923.130005,923.130005,0
1971-04-30,955.250000,955.250000,955.250000,955.250000,955.250000,0
...,...,...,...,...,...,...
2015-09-02,20558.400391,20558.400391,20558.400391,20558.400391,20558.400391,0
2015-09-03,20590.599609,20590.599609,20590.599609,20590.599609,20590.599609,0
2015-09-04,20304.199219,20304.199219,20304.199219,20304.199219,20304.199219,0


**Step 3: Results: Total 37 Equity Indexes**

*1. Pass First Test:*

lst_1 = [('^GSPC', '^DJU'),
 ('^GSPC', '^VIX'),
 ('^RUI', '^DJU'),
 ('^RUI', '^VIX'),
 ('^RUT', '^DJI'),
 ('^RUT', '^DJU'),
 ('^RUT', '^DJA'),
 ('^RUT', '^DJT'),
 ('^RUT', '^VIX'),
 ('^RUT', '^GDAXI'),
<span style='background :yellow' > ('^RUT', '^SP400'),</span> 
 ('^RUT', '^GDAXI'),
 ('^RUT', '^MID'),
 ('^RUT', '^NBI'),
 ('^RUT', '^XMI'),
 ('^RUA', '^DJI'),
 ('^RUA', '^DJU'),
 ('^RUA', '^VIX'),
 ('^DJI', '^DJU'),
 ('^DJI', '^VIX'),
 ('^DJI', '^W5000'),
 ('^DJU', '^DJA'),
 ('^DJU', '^DJT'),
 ('^DJU', '^VIX'),
 ('^DJU', '^NYA'),
 ('^DJU', '^FTSE'),
 ('^DJU', '^GDAXI'),
 ('^DJU', '^FTSE'),
 ('^DJU', '^SP400'),
 ('^DJU', '^SP600'),
 ('^DJU', '^SP1500'),
 ('^DJU', '^GDAXI'),
 ('^DJU', '^BVSP'),
 ('^DJU', '^GSPTSE'),
 ('^DJU', '^MID'),
 ('^DJU', '^W5000'),
 ('^DJU', '^AXJO'),
 ('^DJU', '^GSPTSE'),
 ('^DJU', '^NBI'),
 ('^DJU', '^XMI'),
 ('^DJU', '^BVSP'),
 ('^DJA', '^DJT'),
 ('^DJA', '^DWCF'),
 ('^DJA', '^VIX'),
 ('^DJA', '^GDAXI'),
 ('^DJA', '^SP400'),
 ('^DJA', '^SP600'),
 ('^DJA', '^GDAXI'),
 ('^DJA', '^MID'),
 ('^DJA', '^W5000'),
 ('^DJA', '^NBI'),
 ('^DJT', '^VIX'),
 ('^DJT', '^GDAXI'),
 ('^DJT', '^SP400'),
 ('^DJT', '^SP600'),
 ('^DJT', '^GDAXI'),
 ('^DJT', '^MID'),
 ('^DJT', '^W5000'),
 ('^DJT', '^NBI'),
 ('^DJT', '^XMI'),
 ('^DWCF', '^IXIC'),
 ('^DWCF', '^VIX'),
 ('^DWCF', '^GDAXI'),
 ('^DWCF', '^GDAXI'),
 <span style='background :yellow' >('^DWCF', '^W5000'),</span> 
 ('^IXIC', '^VIX'),
 ('^VIX', '^N225'),
 ('^NYA', '^GDAXI'),
 ('^NYA', '^GDAXI'),
 ('^NYA', '^GSPTSE'),
 ('^NYA', '^GSPTSE'),
 ('^HSI', '^XAU'),
 ('^HSI', '^MXX'),
 ('^GDAXI', '^XAU'),
 ('^GDAXI', '^NBI'),
 ('^GDAXI', '^XMI'),
 ('^N225', '^XAU'),
 ('MSCI', '^SOX'),
 ('MSCI', '^BVSP'),
 ('MSCI', '^GSPTSE'),
 ('MSCI', '^AXJO'),
 ('MSCI', '^GSPTSE'),
 ('MSCI', '^BVSP'),
 ('^SP400', '^SP600'),
 ('^SP400', '^NBI'),
 ('^SP600', '^MID'),
 ('^SP600', '^NBI'),
 ('^GDAXI', '^XAU'),
 ('^GDAXI', '^NBI'),
 ('^GDAXI', '^XMI'),
 ('^GSPTSE', '^AXJO'),
 ('^GSPTSE', '^BVSP'),
 ('^MID', '^NBI'),
 ('^AXJO', '^GSPTSE'),
 ('^GSPTSE', '^BVSP'),
 ('^NBI', '^XMI'),
 ('^MXX', '^KS11')]



*2. Pass Second Test:*


lst_2 = [('^GSPC', '^DJU'),
 ('^RUI', '^DJU'),
 ('^RUT', '^DJI'),
 ('^RUT', '^DJU'),
 ('^RUT', '^DJA'),
 ('^RUT', '^DJT'),
 ('^RUT', '^GDAXI'),
 <span style='background :yellow' >('^RUT', '^SP400'),</span> 
 ('^RUT', '^GDAXI'),
 ('^RUT', '^MID'),
 ('^RUT', '^NBI'),
 ('^RUT', '^XMI'),
 ('^RUA', '^DJU'),
 ('^RUA', '^SP1500'),
 ('^DJI', '^DJU'),
 ('^DJI', '^W5000'),
 ('^DJU', '^DJA'),
 ('^DJU', '^DJT'),
 ('^DJU', '^NYA'),
 ('^DJU', '^GDAXI'),
 ('^DJU', '^SP400'),
 ('^DJU', '^SP600'),
 ('^DJU', '^SP1500'),
 ('^DJU', '^GDAXI'),
 ('^DJU', '^BVSP'),
 ('^DJU', '^GSPTSE'),
 ('^DJU', '^MID'),
 ('^DJU', '^W5000'),
 ('^DJU', '^GSPTSE'),
 ('^DJU', '^NBI'),
 ('^DJU', '^XMI'),
 ('^DJU', '^BVSP'),
 ('^DJA', '^DJT'),
 ('^DJA', '^DWCF'),
 ('^DJA', '^GDAXI'),
 ('^DJA', '^SP400'),
 ('^DJA', '^SP600'),
 ('^DJA', '^GDAXI'),
 ('^DJA', '^MID'),
 ('^DJA', '^NBI'),
 ('^DJT', '^GDAXI'),
 ('^DJT', '^SP400'),
 ('^DJT', '^SP600'),
 ('^DJT', '^GDAXI'),
 ('^DJT', '^MID'),
 ('^DJT', '^W5000'),
 ('^DJT', '^NBI'),
 ('^DWCF', '^IXIC'),
 ('^DWCF', '^GDAXI'),
 ('^DWCF', '^GDAXI'),
<span style='background :yellow' > ('^DWCF', '^W5000'),</span> 
 ('^VIX', '^NYA'),
 ('^VIX', '^NDX'),
 ('^VIX', '^FTSE'),
 ('^VIX', '^HSI'),
 ('^VIX', '^GDAXI'),
 ('^VIX', '^N225'),
 ('^VIX', '^FTSE'),
 ('^VIX', 'MSCI'),
 ('^VIX', '^SP400'),
 ('^VIX', '^SP600'),
 ('^VIX', '^SP1500'),
 ('^VIX', '^GDAXI'),
 ('^VIX', '^SOX'),
 ('^VIX', '^BVSP'),
 ('^VIX', '^GSPTSE'),
 ('^VIX', '^MID'),
 ('^VIX', '^W5000'),
 ('^VIX', '^XAU'),
 ('^VIX', '^AXJO'),
 ('^VIX', '^GSPTSE'),
 ('^VIX', '^NBI'),
 ('^VIX', '^MERV'),
 ('^VIX', '^XMI'),
 ('^VIX', '^BVSP'),
 ('^VIX', '^MXX'),
 ('^VIX', '^KS11'),
 ('^NYA', '^GDAXI'),
 ('^NYA', '^GDAXI'),
 ('^NYA', '^GSPTSE'),
 ('^NYA', '^GSPTSE'),
 ('^NYA', '^NBI'),
 ('^FTSE', '^SP1500'),
 ('^FTSE', '^NBI'),
 ('^HSI', '^MXX'),
 ('^HSI', '^KS11'),
 ('^GDAXI', '^NBI'),
 ('^GDAXI', '^XMI'),
 ('^FTSE', '^SP1500'),
 ('^FTSE', '^NBI'),
 ('MSCI', '^SOX'),
 ('MSCI', '^AXJO'),
 ('^SP400', '^SP600'),
 ('^SP400', '^NBI'),
 ('^SP600', '^MID'),
 ('^SP600', '^NBI'),
 ('^GDAXI', '^NBI'),
 ('^GDAXI', '^XMI'),
 ('^BVSP', '^GSPTSE'),
 ('^BVSP', '^GSPTSE'),
 ('^GSPTSE', '^AXJO'),
 ('^MID', '^NBI'),
 ('^AXJO', '^GSPTSE'),
 ('^NBI', '^XMI'),
 ('^MXX', '^KS11')]

*3. Pass Both Test (Excluding DWCF since cannot download most recent data):*

lst = [('^GSPC', '^DJU'),
 ('^RUI', '^DJU'),
 ('^RUT', '^DJI'),
 ('^RUT', '^DJU'),
 ('^RUT', '^DJA'),
 ('^RUT', '^DJT'),
 ('^RUT', '^GDAXI'),
<span style='background :yellow' > ('^RUT', '^SP400'),</span>
 ('^RUT', '^GDAXI'),
 ('^RUT', '^MID'),
 ('^RUT', '^NBI'),
 ('^RUT', '^XMI'),
 ('^RUA', '^DJU'),
 ('^DJI', '^DJU'),
 ('^DJI', '^W5000'),
 ('^DJU', '^DJA'),
 ('^DJU', '^DJT'),
 ('^DJU', '^NYA'),
 ('^DJU', '^GDAXI'),
 ('^DJU', '^SP400'),
 ('^DJU', '^SP600'),
 ('^DJU', '^SP1500'),
 ('^DJU', '^GDAXI'),
 ('^DJU', '^BVSP'),
 ('^DJU', '^GSPTSE'),
 ('^DJU', '^MID'),
 ('^DJU', '^W5000'),
 ('^DJU', '^GSPTSE'),
 ('^DJU', '^NBI'),
 ('^DJU', '^XMI'),
 ('^DJU', '^BVSP'),
 ('^DJA', '^DJT'),
 ('^DJA', '^DWCF'),
 ('^DJA', '^GDAXI'),
 ('^DJA', '^SP400'),
 ('^DJA', '^SP600'),
 ('^DJA', '^GDAXI'),
 ('^DJA', '^MID'),
 ('^DJA', '^NBI'),
 ('^DJT', '^GDAXI'),
 ('^DJT', '^SP400'),
 ('^DJT', '^SP600'),
 ('^DJT', '^GDAXI'),
 ('^DJT', '^MID'),
 ('^DJT', '^W5000'),
 ('^DJT', '^NBI'),
 ('^VIX', '^N225'),
 ('^NYA', '^GDAXI'),
 ('^NYA', '^GDAXI'),
 ('^NYA', '^GSPTSE'),
 ('^NYA', '^GSPTSE'),
 ('^HSI', '^MXX'),
 ('^GDAXI', '^NBI'),
 ('^GDAXI', '^XMI'),
 ('MSCI', '^SOX'),
 ('MSCI', '^AXJO'),
 ('^SP400', '^SP600'),
 ('^SP400', '^NBI'),
 ('^SP600', '^MID'),
 ('^SP600', '^NBI'),
 ('^GDAXI', '^NBI'),
 ('^GDAXI', '^XMI'),
 ('^GSPTSE', '^AXJO'),
 ('^MID', '^NBI'),
 ('^AXJO', '^GSPTSE'),
 ('^NBI', '^XMI'),
 ('^MXX', '^KS11')]

*4. Final Selected Pairs:*

(1) <span style='background :yellow' >DWCF and W5000</span> 

(2) <span style='background :yellow' >RUT and S&P 400</span> 